Анализа тональности отзывов на примере сентимент-анализа отзывов на фильмы из стандартного датасета nltk.

In [7]:
import warnings
from nltk.corpus import movie_reviews
import nltk
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer 
warnings.simplefilter('ignore')

id негативных и позитивных отзывов:

In [8]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

Список негативных отзывов:

In [9]:
negreviews = [movie_reviews.words(fileids=[f]) for f in negids]
posreviews  = [movie_reviews.words(fileids=[f]) for f in posids]

In [10]:
print 'Негативных отзывов {} штук'.format(len(negreviews))

Негативных отзывов 1000 штук


In [11]:
print 'Позитивных отзывов отзывов {} штук'.format(len(posreviews))

Позитивных отзывов отзывов 1000 штук


negreviews - list содержащий отзывы, каждый отзыв - list элементами которого являются отдельные слова отзыва.
CountVectorizer на вход принимет  лист содержащий отзывы в виде строк, поэтому преобразовываем исходные даннные. 

In [12]:
negreviews[1]

[u'the', u'happy', u'bastard', u"'", u's', u'quick', ...]

In [13]:
negreviews_str =[' '.join(review) for review in negreviews]

In [14]:
posreviews_str =[' '.join(review) for review in posreviews]

In [15]:
texts = negreviews_str + posreviews_str
labels = [0] * len(negreviews_str) + [1] * len(posreviews_str)

Используем CountVectorizer с настройками по умолчанию для того, чтобы получить признаковое представление каждого текста.

In [16]:
CV = CountVectorizer() 

In [17]:
CV.fit_transform(texts)

<2000x39659 sparse matrix of type '<type 'numpy.int64'>'
	with 666842 stored elements in Compressed Sparse Row format>

In [18]:
X = CV.fit_transform(texts).toarray()

In [19]:
def get_frequent_words(X, vocabulary):
    Frequencys = {}
    invert_dict= {}
    for key in vocabulary.keys():   
        invert_dict.update({vocabulary.get(key):key}) 
    for i in xrange(0,X.shape[1]):
        Frequencys.update( { invert_dict.get(i) : sum(X[:,i]) } )
    Frequencys = sorted(Frequencys.items(), key=lambda x:x[1],reverse=True)     
    
    return Frequencys

In [21]:
print get_frequent_words(X,CV.vocabulary_)[:20]

[(u'the', 76529), (u'and', 35576), (u'of', 34123), (u'to', 31937), (u'is', 25195), (u'in', 21822), (u'it', 16107), (u'that', 15924), (u'as', 11378), (u'with', 10792), (u'for', 9961), (u'his', 9587), (u'this', 9578), (u'film', 9517), (u'he', 8864), (u'but', 8634), (u'on', 7385), (u'are', 6949), (u'by', 6261), (u'be', 6174)]


pipeline из CountVectorizer и LogisticRegression c настройками по-умолчанию и с помощью cross_val_score (также со стандартными настройками), оценка качества по accuracy и ROC AUC.

In [22]:
def text_classifier(vectorizer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
            ("classifier", classifier)]
        )

In [23]:
print cross_val_score(text_classifier(CountVectorizer(), LogisticRegression()), texts, labels).mean()

0.836021650393


In [24]:
print cross_val_score(text_classifier(CountVectorizer(), LogisticRegression()), texts, labels,scoring='roc_auc').mean()

0.910776493783


Обучение логистической регрессии на всей доступной выборке и опредедение наиболее важных для модели признаков (слов). 

In [25]:
b = text_classifier(CountVectorizer(), LogisticRegression()).fit(texts, labels)
array_of_coef = b.named_steps['classifier'].coef_

In [26]:
# Наиболее значимые, весомые слова 
def major_words(array_of_coef,vocabulary,n_words=20):

#array_of_coef - массив коэффициентов линейной регрессии LogisticRegression().coef_
#vocabulary - словарь CountVectorizer().vocabulary_
#n_words - количество возвращаемых слов
   
    invert_vocabulary = {}
    for key in vocabulary.keys():   
        invert_vocabulary.update({vocabulary.get(key):key})
   
    # Если нужет топ 'ругательств' убрать abs() и поменять reverse=False
    major_words= {}
    for g in xrange(0,len(array_of_coef[0])):
         major_words.update({g: abs(array_of_coef[0][g])}) 
        #major_words.update({g: array_of_coef[0][g]}) 
        
    major_words = sorted(major_words.items(), key=lambda x:x[1],reverse=True)   
    #major_words = sorted(major_words.items(), key=lambda x:x[1])   
    
    array_of_words=[]
    for i in xrange(0,n_words):
        array_of_words.append(invert_vocabulary.get(major_words[i][0]))
        
    
    return array_of_words

In [27]:
major_words(array_of_coef,CV.vocabulary_)

[u'bad',
 u'unfortunately',
 u'worst',
 u'fun',
 u'waste',
 u'nothing',
 u'script',
 u'awful',
 u'boring',
 u'only',
 u'great',
 u'plot',
 u'poor',
 u'reason',
 u'back',
 u'looks',
 u'mess',
 u'supposed',
 u'lame',
 u'quite']

       1. Здесь и далее оценка качества будет выполняться с помощью cross_val_score с cv=5 и остальными параметрами по умолчанию. Оценено среднее качество ( .mean() ) и стандартное отклонение (.std()) по fold'ам для:
        а) pipeline из CountVectorizer() и LogisticRegression(),
        б) pipeline из TfidfVectorizer() и LogisticRegression(). 

        2. Изменяем параметр min_df (min document frequency) у CountVectorizer: min_df=10 и с min_df=50.
        
        3. Меняем классификатор, используемый после CountVectorizer. Сравниваем результаты для LogisticRegression, LinearSVC и SGDClassifier. 
        
        4. Используем список стоп-слов с помощью nltk.corpus.stopwords.words('english') и собственный список стоп-слов sklearn. 
        
        5. Добавляем в CountVectorizer биграммы.
        
        6. Строим модель на частотах буквенных n-грамм c n от 3 до 5, указав соответствующее значение параметра ngram_range и параметр analyzer='char_wb'. 

1.

In [28]:
CVLR = cross_val_score(text_classifier(CountVectorizer(), LogisticRegression()), texts, labels, cv=5)

In [29]:
print CVLR.mean()
print CVLR.std()

0.841
0.0167779617356


In [30]:
TFLR =  cross_val_score(text_classifier(TfidfVectorizer(), LogisticRegression()), texts, labels,cv=5)

In [31]:
print TFLR.mean()
print TFLR.std()

0.821
0.00406201920232


2.

In [37]:
CVLR50 = cross_val_score(text_classifier(CountVectorizer(min_df=50), LogisticRegression()), texts, labels, cv=5)

In [38]:
print CVLR50.mean()
print CVLR50.std()

0.813
0.0134536240471


In [39]:
CVLR10 = cross_val_score(text_classifier(CountVectorizer(min_df=10), LogisticRegression()), texts, labels, cv=5)

In [40]:
print CVLR10.mean()
print CVLR10.std()

0.839
0.0118953772534


3.

In [41]:
CVLR = cross_val_score(text_classifier(CountVectorizer(), LogisticRegression()), texts, labels, cv=5)

In [42]:
print CVLR.mean()
print CVLR.std()

0.841
0.0167779617356


In [43]:
CVLinR = cross_val_score(text_classifier(CountVectorizer(), LinearSVC()), texts, labels, cv=5)

In [44]:
print CVLinR.mean()
print CVLinR.std()

0.8325
0.0162788205961


In [45]:
CVSGDC = cross_val_score(text_classifier(CountVectorizer(), SGDClassifier()), texts, labels, cv=5)

In [46]:
print CVSGDC.mean()
print CVSGDC.std()

0.7545
0.111740771431


4.

In [49]:
stopwords = nltk.corpus.stopwords.words('english')

In [50]:
CVLRstop_1 = cross_val_score(text_classifier(CountVectorizer(stop_words=stopwords), LogisticRegression()), texts, labels, cv=5)

In [51]:
print CVLRstop_1.mean()
print CVLRstop_1.std()

0.8415
0.0104403065089


In [52]:
CVLRstop_2 = cross_val_score(text_classifier(CountVectorizer(stop_words='english'), LogisticRegression()), texts, labels, cv=5)

In [53]:
print CVLRstop_2.mean()
print CVLRstop_2.std()

0.8385
0.00982344135219


5.

In [54]:
CVLR_2 = cross_val_score(text_classifier(CountVectorizer(ngram_range=(1, 2)), LogisticRegression()), texts, labels, cv=5)

In [55]:
print CVLR_2.mean()
print CVLR_2.std()

0.8525
0.0165075740192


In [57]:
CVLR_35 = cross_val_score(text_classifier(CountVectorizer(ngram_range=(3, 5),analyzer='char_wb'), LogisticRegression()), texts, labels, cv=5)

In [58]:
print CVLR_35.mean()
print CVLR_35.std()

0.82
0.0106066017178
